In [1]:
#!pip3 install botocore boto3 ujson warcio

In [2]:
harpercollins_cc_url = "crawl-data/CC-MAIN-2020-24/segments/1590348509972.80/crawldiagnostics/CC-MAIN-20200606031557-20200606061557-00532.warc.gz"
harpercollins_cc_url = "https://commoncrawl.s3.amazonaws.com/" + harpercollins_cc_url
print(harpercollins_cc_url)

https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-24/segments/1590348509972.80/crawldiagnostics/CC-MAIN-20200606031557-20200606061557-00532.warc.gz


In [3]:
from tools.datasets import *

In [4]:
import os
import fnmatch

def fetch_commoncrawl_dataset(domain: str, _format="json", force_redownload=False):
    """Download and cache a commoncrawl dataset.
    
    By now this method uses classical HTTPS downloads. One may wish to use S3 in the future.
    
    """
    
    download_dir = "commoncrawl"
    os.makedirs(get_filename_path(download_dir), exist_ok=True)
    download_dir_content = os.listdir(get_filename_path(download_dir))
    pattern = "*" + domain + ".*"
    for file in download_dir_content:
        if fnmatch.fnmatch(file, pattern):
            print(f"Dataset {file} was found. Skipping...")
            return os.path.join(download_dir, file)
    
    print("No matching dataset was found. Querying commoncrawl...")
    query_url = "https://index.commoncrawl.org/CC-MAIN-2020-24-index?url=" + domain + "&output=json"
    query = requests.get(query_url)
    query.raise_for_status()
    # Despite the wished return format, the call does not return a valid JSON result.
    results = query.content.decode("utf-8").split("\n")[1]

    result = json.loads(results)
    original_resource_path = result["filename"]
    filename = os.path.split(original_resource_path)[1]
    filename_stripped, filename_ext = os.path.splitext(filename)
    
    print("Found matching result " + filename)
    
    complete_url = "https://commoncrawl.s3.amazonaws.com/" + original_resource_path
    save_path = f"{download_dir}/{filename_stripped}-{domain}{filename_ext}"
    download_to(complete_url, save_path)
    print("Done")
    return save_path
    

In [5]:
collins_gz = fetch_commoncrawl_dataset("collinsdictionary.com")

No matching dataset was found. Querying commoncrawl...
Found matching result CC-MAIN-20200606031557-20200606061557-00059.warc.gz
Done


In [8]:
from warcio.archiveiterator import ArchiveIterator

with wrap_open(collins_gz, handler=gzip.open) as stream:
    for record in ArchiveIterator(stream):
        if record.rec_type == "response":
            if record.http_headers.get_header('Content-Type') == 'text/html':
                target_uri = record.rec_headers.get_header('WARC-target-URI')
                if "collins" in target_uri:                    
                    print(record.rec_headers.get_header('WARC-Target-URI'))
                    print('')



http://025.839214.com/20191203/24.html

http://0515ycnk.com/rmtzgjyb/201910/t20191027_270052.html

http://079133.com/article.php?id=109

http://079133.com/category.php?id=73

http://0805.asianbusinessav.com/

http://0qmqkkac.cctcoc.org.cn/ckuo7e/qgu7we/237256/

http://1-a-odejda.ru/product_info.php?cPath=45_49&products_id=197&osCsid=3b6534bd4c67baaf02fac6541df940b0

http://106jsb.com/list.asp?id=3301

http://1122jecom.cn/2crshi/bpsc-hindi-sahitya-syllabus.html

http://123123456com.com/list.php?383.html

http://171bm.space/?/sell/32621

http://20dar20.com/143899-%D8%A2%D9%BE%D8%A7%D8%B1%D8%AA%D9%85%D8%A7%D9%86-%D8%B3%D9%87-%D9%86%D8%A8%D8%B4-%D8%B3%D9%87-%D8%AE%D9%88%D8%A7%D8%A8-%D8%A8%D9%87-%D9%85%D8%AA%D8%B1%D8%A7%DA%98-124-%D9%85%D8%AA%D8%B1-%D9%85%D8%B1%D8%A8%D8%B9.html

http://21cngolf.com/21cngolf_181076_56541_297943/

http://245.shkaoyan.com/20191203/6269.html

http://285creative.com.au/teamwear.php

http://2gowh.beimeishijia.com/243.xml

http://3g.bsteel.net.cn:9245/

http://3g.

In [14]:
print(collins_content[:10000])


b'WARC/1.0\r\nWARC-Type: warcinfo\r\nWARC-Date: 2020-06-06T03:15:57Z\r\nWARC-Record-ID: <urn:uuid:962190be-a294-4252-982e-c8d0601f8fab>\r\nContent-Length: 501\r\nContent-Type: application/warc-fields\r\nWARC-Filename: CC-MAIN-20200606031557-20200606061557-00532.warc.gz\r\n\r\nisPartOf: CC-MAIN-2020-24\r\npublisher: Common Crawl\r\ndescription: Wide crawl of the web for May/June 2020\r\noperator: Common Crawl Admin (info@commoncrawl.org)\r\nhostname: ip-10-67-67-5.ec2.internal\r\nsoftware: Apache Nutch 1.16 (modified, https://github.com/commoncrawl/nutch/)\r\nrobots: checked via crawler-commons 1.1-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)\r\nformat: WARC File Format 1.1\r\nconformsTo: http://iipc.github.io/warc-specifications/specifications/warc-format/warc-1.1/\r\n\r\n\r\nWARC/1.0\r\nWARC-Type: request\r\nWARC-Date: 2020-06-06T05:21:07Z\r\nWARC-Record-ID: <urn:uuid:767322c8-b62b-41fc-abbb-9c155aab02fd>\r\nContent-Length: 308\r\nContent-Type: application/http; msgty

In [17]:
import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

3.14164112


In [28]:
from pyspark.sql import SparkSession

session = SparkSession.builder.config(conf=sc.getConf()).getOrCreate()
df = session.read.load('s3://commoncrawl/cc-index/table/cc-main/warc')
df.createOrReplaceTempView('ccindex')

Py4JJavaError: An error occurred while calling o308.load.
: java.io.IOException: No FileSystem for scheme: s3
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:547)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scala:545)
	at scala.collection.TraversableLike$$anonfun$f

In [ ]:
from warcio.archiveiterator import ArchiveIterator
from io import BytesIO

import warcio

